# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import pandas as pd
import numpy as np
import pickle #pickle
from sklearn.externals import joblib
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [58]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName",engine)
X = df.message.values
Y = df.iloc[:,4:].values
#

In [59]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [60]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [61]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [63]:
y_pred = pipeline.predict(X_test)

In [64]:
for i,name in enumerate(df.iloc[:,4:].columns):
    print(name)
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.57      0.47      0.52      1463
          1       0.86      0.90      0.88      5091

avg / total       0.79      0.80      0.80      6554

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5410
          1       0.85      0.35      0.50      1144

avg / total       0.87      0.88      0.85      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.69      0.91      0.78      3796
          1       0.77      0.43      0.55      2758

avg / total       0.72      0.71      0.68      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/home/fafun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [69]:
 parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000,),
        'tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100],
        'clf__min_samples_split': [2, 3],
    }


cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [70]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [74]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=5000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [75]:
cv.best_params_

{'clf__min_samples_split': 2,
 'clf__n_estimators': 100,
 'tfidf__use_idf': False,
 'vect__max_df': 0.5,
 'vect__max_features': 5000,
 'vect__ngram_range': (1, 2)}

In [71]:
for i,name in enumerate(df.iloc[:,4:].columns):
    print(name)
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.67      0.45      0.54      1463
          1       0.86      0.94      0.89      5091

avg / total       0.82      0.83      0.82      6554

request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5410
          1       0.89      0.47      0.61      1144

avg / total       0.90      0.90      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.92      0.80      3796
          1       0.81      0.50      0.62      2758

avg / total       0.76      0.74      0.73      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/home/fafun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [85]:
#add other features besides the TF-IDF
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])
    
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), ),
        'features__text_pipeline__vect__max_df': (0.5, ),
        'features__text_pipeline__vect__max_features': (5000,),
        'features__text_pipeline__tfidf__use_idf': (False,),
        'clf__n_estimators': [100,],
        'clf__min_samples_split': [2,],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},)
    }


    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

[nltk_data] Downloading package punkt to /home/fafun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/fafun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fafun/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [86]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
for i,name in enumerate(df.iloc[:,4:].columns):
    print(name)
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.69      0.41      0.52      1463
          1       0.85      0.95      0.90      5091

avg / total       0.81      0.83      0.81      6554

request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5410
          1       0.89      0.46      0.61      1144

avg / total       0.90      0.90      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.91      0.81      3796
          1       0.81      0.52      0.63      2758

avg / total       0.76      0.75      0.73      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/home/fafun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [72]:
# save the model as pickle file
with open('cv.pickle', 'wb') as f:
    pickle.dump(cv, f)

In [78]:
#load the pickle file
model = joblib.load("cv.pickle")

In [79]:
#test the loaded model
y_pred = model.predict(X_test)
for i,name in enumerate(df.iloc[:,4:].columns):
    print(name)
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.67      0.45      0.54      1463
          1       0.86      0.94      0.89      5091

avg / total       0.82      0.83      0.82      6554

request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5410
          1       0.89      0.47      0.61      1144

avg / total       0.90      0.90      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.92      0.80      3796
          1       0.81      0.50      0.62      2758

avg / total       0.76      0.74      0.73      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/home/fafun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.